In [1]:
import datetime
import logging
import os
import glob
import pandas as pd

from utils.dialog_manipulation import prepare_dialogs_sorted_by_lang
from utils.dialog_manipulation import add_reply_time, add_subdialogs_ids

In [14]:
# def init_args():
#     parser = argparse.ArgumentParser(description="Step #3.Prepare dialogs data.")
    
#     parser.add_argument(
#         "--dialogs_ids",
#         nargs="+",
#         type=int,
#         help="id(s) of dialog(s) to download, -1 for all",
#         required=True,
#     )
#     parser.add_argument(
#         "--config_path",
#         type=str,
#         help="path to config file",
#         default="config/config.json",
#     )
#     parser.add_argument("--debug_mode", type=int, help="Debug mode", default=0)
#     return parser.parse_args()

## Change all this block to your parameters

In [8]:
# args = init_args()


# DIALOGS_IDS = args.dialogs_ids
# DEBUG_MODE = args.debug_mode


# input dialogs_ids (it should be a list) or -1 for all
DIALOGS_IDS = [-1]
DEBUG_MODE = 0
DIALOG_PATH = os.path.join('data', 'test_dialogs')
LOGS_PATH = 'logs/project_logs.log'

FINAL_DF_NAME = "all.csv"
PATH_TO_PREPARED_DIALOGS = os.path.join("data", "dialogs_prepared")
PATH_TO_SAVE_GENERAL_DF = os.path.join("data", "dialogs_prepared", FINAL_DF_NAME)

# change to date range in what you want to analyse messages of user_id_get_msg - from START_DATE to END_DATE;
# format "%Y-%m-%d %H:%M:%S"
START_DATE = datetime.datetime(2020, 8, 9, 0, 0, 0)
END_DATE = datetime.datetime(2020, 8, 10, 0, 0, 0)

In [5]:
# !pip install -r requirements.txt
# !pip install -U nlpcube

In [17]:
if DEBUG_MODE:
    logging.basicConfig(filename=LOGS_PATH, level=logging.DEBUG)

if os.path.isdir(DIALOG_PATH):
    if not os.path.isdir(PATH_TO_PREPARED_DIALOGS):
        os.mkdir(PATH_TO_PREPARED_DIALOGS)
    
    prepare_dialogs_sorted_by_lang(DIALOGS_IDS, DIALOG_PATH, PATH_TO_PREPARED_DIALOGS,
                                   START_DATE, END_DATE, "add_lang_column")

else:
    logging.error('Dialogs dir does not exist !')

[-1]
dialog_ids_sorted_by_lang
{'en': [], 'ru': ['-169766999'], 'ua': ['-193231004', '-197859086']}
Loading latest local model: uk-1.1
	Loading embeddings ... 
	Loading tokenization model ...
	Loading lemmatization model ...
	Loading tagger model ...
	Loading parser model ...


Model loading complete.


=======Language ua -- 1 from 3=======

=======Language ua -- 2 from 3=======

=======Language ru -- 3 from 3=======
INDEX 0 from 454
INDEX 1 from 454
INDEX 2 from 454
INDEX 4 from 454
INDEX 5 from 454
INDEX 6 from 454
INDEX 7 from 454
INDEX 9 from 454
INDEX 10 from 454
INDEX 11 from 454
INDEX 12 from 454
INDEX 18 from 454
INDEX 19 from 454
INDEX 20 from 454
INDEX 21 from 454
INDEX 26 from 454
INDEX 27 from 454
INDEX 28 from 454
INDEX 38 from 454
INDEX 39 from 454
INDEX 40 from 454
INDEX 41 from 454
INDEX 42 from 454
INDEX 43 from 454
INDEX 44 from 454
INDEX 45 from 454
INDEX 46 from 454
INDEX 47 from 454
INDEX 48 from 454
INDEX 49 from 454
INDEX 50 from 454
INDEX 51 from 454
INDEX 52 from 454
INDEX 53 from 454
INDEX 54 from 454
INDEX 55 from 454
INDEX 56 from 454
INDEX 57 from 454
INDEX 60 from 454
INDEX 61 from 454
INDEX 62 from 454
INDEX 63 from 454
INDEX 65 from 454
INDEX 66 from 454
INDEX 67 from 454
INDEX 69 from 454
INDEX 70 from 454
INDEX 71 from 454
IN

INDEX 115 from 454


In [10]:
flag_get_all = 0

if DIALOGS_IDS[0] == -1:
    DIALOGS_IDS = glob.glob(f"{PATH_TO_PREPARED_DIALOGS}/*.csv")
    flag_get_all = 1

In [11]:
DIALOGS_IDS

['data/dialogs_prepared/-169766999.csv',
 'data/dialogs_prepared/-193231004.csv',
 'data/dialogs_prepared/all.csv',
 'data/dialogs_prepared/-197859086.csv']

In [13]:
frames = []

general_df = pd.DataFrame()
general_n_subdialogs = 0
general_dialog_id = 0
len_dialogs = len(DIALOGS_IDS)

for n_dialog_id, dialog_file in enumerate(DIALOGS_IDS):
    dialog_id = dialog_file.split('/')[-1]
    
    if dialog_id == FINAL_DF_NAME:
        continue
    
    if flag_get_all == 1:
        dialog_id = str(dialog_id)[:-4]

    data = pd.read_csv(os.path.join(PATH_TO_PREPARED_DIALOGS, f"{dialog_id}.csv"))

    add_reply_time(data)
    add_subdialogs_ids(data)

    data["dialog ID"] = n_dialog_id

    general_df = pd.concat([general_df, data])
    print(f"{n_dialog_id + 1} dialogs csv from {len_dialogs} succeeded")


general_df.rename(columns={'id': 'message_id'}, inplace=True)

general_df = general_df.drop("Unnamed: 0", axis=1)
general_df = general_df.drop("Unnamed: 0.1", axis=1)

cols = ['dialog ID', "message_id", "date", "from_id", 'to_id', 'fwd_from', 'dialog_language',
        'reply_time', 'subdialog_id', 'message', 'preprocessed_message']

general_df = general_df[cols]

/Users/akurochkin/main/project/python/telegram-dialogs-analysis/utils/dialog_manipulation.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["reply_time"][index] = 0
/Users/akurochkin/main/project/python/telegram-dialogs-analysis/utils/dialog_manipulation.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["reply_time"][index] = time_diff.total_seconds()
/Users/akurochkin/main/project/python/telegram-dialogs-analysis/utils/dialog_manipulation.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas

1 dialogs csv from 4 succeeded
2 dialogs csv from 4 succeeded
4 dialogs csv from 4 succeeded


In [14]:
general_df.to_csv(PATH_TO_SAVE_GENERAL_DF, index=False)